# Introduction

What are optimizers?





In [45]:
!pip -q install jupyter_bokeh

### Must watch from TEST and FINAL's perspective

  - Momentum : https://www.youtube.com/watch?v=Vce8w1sy0e8
  - RMSProp  : https://www.youtube.com/watch?v=ajI_HTyaCu8&t=102s
  - Adam     : https://www.youtube.com/watch?v=tuU59-G1PgU




# Defining Loss Terrain

In [46]:
import torch
import torch.nn.functional as F

class LossFunction:
    def __init__(self, A_g, sigma_g, global_min_pos, local_min_params, noise_factor=0.1):
        self.A_g = A_g
        self.sigma_g = sigma_g
        self.global_min_pos = global_min_pos
        self.local_min_params = local_min_params
        self.noise_factor = noise_factor

    def generate_terrain_mesh(self, x_range, y_range):
        """Generates a terrain mesh."""
        # Create a meshgrid of x and y values using PyTorch
        x = torch.linspace(x_range[0], x_range[1], 100)
        y = torch.linspace(y_range[0], y_range[1], 100)
        X, Y = torch.meshgrid(x, y, indexing='ij')

        # Return the loss calculated at each (X, Y) point
        return X, Y, self.get_loss(X, Y)

    def get_loss(self, X, Y):
        """Calculates the loss."""
        # Global minimum: Gaussian function
        Z = self.A_g * torch.exp(-((X - self.global_min_pos[0])**2 + (Y - self.global_min_pos[1])**2) / (2 * self.sigma_g**2))

        # Local minima: Sum of Gaussian functions
        for A_l, sigma_l, x_l, y_l in self.local_min_params:
            Z += A_l * torch.exp(-((X - x_l)**2 + (Y - y_l)**2) / (2 * sigma_l**2))

        # Add random noise
        Z += self.noise_factor * torch.randn_like(X)

        return Z

# Visualizing the terrain

In [82]:
import numpy as np
import plotly.graph_objs as go
import param
import panel as pn
import torch

# Enable Panel extension for Plotly
pn.extension('plotly')

class TerrainModel(param.Parameterized):
    # Define parameters for the model
    x_min = param.Number(-10, bounds=(-20, 0), doc="Minimum x-value")
    x_max = param.Number(10, bounds=(0, 20), doc="Maximum x-value")
    y_min = param.Number(-10, bounds=(-20, 0), doc="Minimum y-value")
    y_max = param.Number(10, bounds=(0, 20), doc="Maximum y-value")

    A_g = param.Number(-10, bounds=(-20, 0), doc="Amplitude of global minimum")
    sigma_g = param.Number(2, bounds=(0.1, 5), doc="Aperture size of global minimum")
    global_min_x = param.Number(0, bounds=(-10, 10), doc="x-position of global minimum")
    global_min_y = param.Number(0, bounds=(-10, 10), doc="y-position of global minimum")

    # Input box for local minima as list of tuples
    local_min_params = param.String("(-5, 1, 3, 3), (-7, 1.5, -4, -4), (-4, 0.8, -2, 5)", doc="Local minima parameters as list of tuples")

    # Set the step size for noise_factor to 0.01
    noise_factor = param.Number(0.00, bounds=(0, 1), step=0.01, doc="Noise factor")

    # Update button
    update_plot = param.Action(lambda x: x.param.trigger('update_plot'), label="Update Plot")

    def parse_local_min_params(self):
        """
        Parse the input string for local minima parameters and return as a list of tuples.
        """
        try:
            # Safely evaluate the input string to convert it into a list of tuples
            local_min_params = eval(self.local_min_params)
            # Ensure it's a list of tuples
            if isinstance(local_min_params, tuple):
                local_min_params = [*local_min_params]
            return local_min_params
        except:
            return []

    @param.depends('update_plot')
    def generate_plot(self):
        # Define the ranges for x and y
        x_range = (self.x_min, self.x_max)
        y_range = (self.y_min, self.y_max)

        # Define global minimum position
        global_min_pos = (self.global_min_x, self.global_min_y)

        # Parse the local minima parameters from the string input
        local_min_params = self.parse_local_min_params()

        if not local_min_params:
            return go.Figure().update_layout(title="Invalid local minima parameters")

        # Generate mesh using the updated parameters
        terrain_loss = LossFunction(
            A_g=-10,
            sigma_g=2,
            global_min_pos=(0, 0),
            local_min_params=[(-5, 1, 3, 3), (-7, 1.5, -4, -4), (-4, 0.8, -2, 5)],
            noise_factor=0.0
        )

        X, Y, Z = terrain_loss.generate_terrain_mesh(x_range, y_range)
        X, Y, Z = X.numpy(), Y.numpy(), Z.numpy()

        # Create the surface plot (3D)
        surface = go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')

        # Create the figure and add the surface (3D plot)
        fig_3d = go.Figure(data=[surface])
        fig_3d.update_layout(scene=dict(
                            xaxis_title='X Axis',
                            yaxis_title='Y Axis',
                            zaxis_title='Z Axis'),
                          title="3D Terrain-like Mesh Plot")

        # Create the contour plot (2D)
        contour = go.Contour(z=Z.flatten(), x=X.flatten(), y=Y.flatten(), colorscale='Viridis', contours=dict(showlabels=True))

        # Create the figure for the contour plot
        fig_contour = go.Figure(data=[contour])
        fig_contour.update_layout(
            title="Contour Plot",
            xaxis_title="X Axis",
            yaxis_title="Y Axis"
        )

        # Return both plots in a Panel layout
        return pn.Row(fig_3d, fig_contour)

# Instantiate the TerrainModel
terrain_model = TerrainModel()

# Create Panel dashboard with dynamic controls and live plot update, now with 3 columns
pn.Row(terrain_model.param, terrain_model.generate_plot).servable()

Row
    [0] Column(margin=(5, 10), name='TerrainModel')
        [0] StaticText(value='<b>TerrainModel</b>')
        [1] FloatSlider(end=0, name='X min', start=-20, value=-10)
        [2] FloatSlider(end=20, name='X max', value=10)
        [3] FloatSlider(end=0, name='Y min', start=-20, value=-10)
        [4] FloatSlider(end=20, name='Y max', value=10)
        [5] FloatSlider(end=0, name='A g', start=-20, value=-10)
        [6] FloatSlider(end=5, name='Sigma g', start=0.1, value=2)
        [7] FloatSlider(end=10, name='Global min x', start=-10)
        [8] FloatSlider(end=10, name='Global min y', start=-10)
        [9] TextInput(description='Local minima p..., name='Local min params', value='(-5, 1, 3, 3), ...)
        [10] FloatSlider(name='Noise factor', step=0.01)
        [11] Button(name='Update Plot')
    [1] ParamMethod(method, _pane=Row, defer_load=False)

# Defining dummy pytorch optization pipeline

In [214]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class TerrainOptimizer(nn.Module):
    def __init__(self, terrain_loss, x0, x1, optimizer=optim.SGD, **optimargs):
        super(TerrainOptimizer, self).__init__()

        # Initialize the two parameters x0, x1 randomly within the x_range and y_range
        self.x0 = nn.Parameter(torch.tensor([x0], dtype=torch.float))
        self.x1 = nn.Parameter(torch.tensor([x1], dtype=torch.float))

        # Store the loss function
        self.terrain_loss = terrain_loss

        # Set up the optimizer
        self.optimizer = optimizer(self.parameters(), **optimargs)

    def forward(self):
        # Forward pass to compute the loss
        X = self.x0.view(-1, 1)  # Reshape to match meshgrid format
        Y = self.x1.view(-1, 1)
        Z = self.terrain_loss.get_loss(X, Y)
        return Z.mean()  # Return the mean loss for optimization

    def optimize(self, log_iter, n_iter=100, verbose = False):
        trace = []  # Store the trace of x0, x1, and loss

        for i in tqdm(range(n_iter)):
            self.optimizer.zero_grad()  # Zero gradients
            loss = self.forward()  # Forward pass
            loss.backward()  # Backward pass
            self.optimizer.step()  # Update parameters
            if i%log_iter == 0:
              # Log the intermediate values
              trace.append((self.x0.item(), self.x1.item(), loss.item()))
              if verbose:
                print(f"\nIteration {i+1}, x0: {self.x0.item():.4f}, x1: {self.x1.item():.4f}, loss: {loss.item():.4f}")

        return trace

# Example usage:
terrain_loss = LossFunction(
    A_g=-10,
    sigma_g=2,
    global_min_pos=(0, 0),
    local_min_params=[(-5, 1, 3, 3), (-7, 1.5, -4, -4), (-4, 0.8, -2, 5)],
    noise_factor=0.0
)

# 3 starting points
EASY_X, EASY_Y = 0.5, 0.5
MODERATE_X, MODERATE_Y = 2.0, 4.0
TOUGH_X, TOUGH_Y = -7.0, 5.0

# Instantiate the optimizer module
model = TerrainOptimizer(
    terrain_loss,
    x0=TOUGH_X,
    x1=TOUGH_Y,
    optimizer=optim.SGD,  # Using different optimizers
    lr=0.1  # Learning rate for the optimizer
)

# Perform optimization
trace = model.optimize(log_iter = 100, n_iter=10000, verbose = True)

  1%|▏         | 136/10000 [00:00<00:14, 675.05it/s]


Iteration 1, x0: -6.9998, x1: 4.9999, loss: -0.0010

Iteration 101, x0: -6.9826, x1: 4.9876, loss: -0.0010


  3%|▎         | 274/10000 [00:00<00:14, 663.05it/s]


Iteration 201, x0: -6.9647, x1: 4.9748, loss: -0.0011

Iteration 301, x0: -6.9459, x1: 4.9614, loss: -0.0011


  5%|▍         | 492/10000 [00:00<00:14, 657.56it/s]

...

 97%|█████████▋| 9688/10000 [00:15<00:00, 709.10it/s]


Iteration 9601, x0: -0.0034, x1: -0.0034, loss: -10.0064

Iteration 9701, x0: -0.0034, x1: -0.0034, loss: -10.0064


 99%|█████████▉| 9900/10000 [00:16<00:00, 670.95it/s]


Iteration 9801, x0: -0.0034, x1: -0.0034, loss: -10.0064

Iteration 9901, x0: -0.0034, x1: -0.0034, loss: -10.0064


100%|██████████| 10000/10000 [00:16<00:00, 618.72it/s]


In [170]:
trace

[(0.4000000059604645, 0.4000000059604645, -9.404646873474121),
 (0.10297776013612747, 0.10297776013612747, -9.971163749694824),
 (0.029842425137758255, 0.029842425137758255, -10.002941131591797),
 (0.007117847446352243, 0.007117848377674818, -10.00601577758789),
 (-5.909023457206786e-05, -5.908834282308817e-05, -10.006321907043457),
 (-0.0023297646548599005, -0.0023297620937228203, -10.006354331970215),
 (-0.003048342652618885, -0.003048339858651161, -10.006356239318848),
 (-0.003275752766057849, -0.003275749972090125, -10.006356239318848),
 (-0.0033477225806564093, -0.003347720019519329, -10.006357192993164),
 (-0.0033704997040331364, -0.003370497142896056, -10.006357192993164)]

# Visualizing the optimization path

In [215]:
import numpy as np
import plotly.graph_objs as go
import param
import panel as pn
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

# Enable Panel extension for Plotly
pn.extension('plotly')

class TerrainModel(param.Parameterized):
    x_min = param.Number(-10, bounds=(-20, 0), doc="Minimum x-value")
    x_max = param.Number(10, bounds=(0, 20), doc="Maximum x-value")
    y_min = param.Number(-10, bounds=(-20, 0), doc="Minimum y-value")
    y_max = param.Number(10, bounds=(0, 20), doc="Maximum y-value")

    A_g = param.Number(-10, bounds=(-20, 0), doc="Amplitude of global minimum")
    sigma_g = param.Number(2, bounds=(0.1, 5), doc="Aperture size of global minimum")
    global_min_x = param.Number(0, bounds=(-10, 10), doc="x-position of global minimum")
    global_min_y = param.Number(0, bounds=(-10, 10), doc="y-position of global minimum")
    local_min_params = param.String("(-5, 1, 3, 3), (-7, 1.5, -4, -4), (-4, 0.8, -2, 5)", doc="Local minima parameters as list of tuples")

    noise_factor = param.Number(0.00, bounds=(0, 1), step=0.01, doc="Noise factor")

    optimizer = param.ObjectSelector(default='SGD', objects=['SGD', 'Adam', 'RMSprop', 'Adagrad', 'AdamW', 'Adadelta',])
    optim_args = param.String('{"lr": 0.1}', doc="Optimizer arguments in Python dict format")
    start_point = param.Tuple((-7.0,5.0), doc="Starting point (x0, y0)")
    n_iter = param.Integer(10000, bounds=(100, 10000), doc="Number of iterations")
    log_iter = param.Integer(100, bounds=(10, 100), doc="Logging interval")

    update_plot = param.Action(lambda x: x.param.trigger('update_plot'), label="Update Plot")

    def parse_local_min_params(self):
        try:
            local_min_params = eval(self.local_min_params)
            if isinstance(local_min_params, tuple):
                local_min_params = [*local_min_params]
            return local_min_params
        except:
            return []

    def parse_optim_args(self):
        try:
            return eval(self.optim_args)
        except:
            return {}

    @param.depends('update_plot')
    def generate_plot(self):
        x_range = (self.x_min, self.x_max)
        y_range = (self.y_min, self.y_max)
        global_min_pos = (self.global_min_x, self.global_min_y)
        local_min_params = self.parse_local_min_params()
        optim_args = self.parse_optim_args()

        if not local_min_params:
            return go.Figure().update_layout(title="Invalid local minima parameters")

        terrain_loss = LossFunction(self.A_g, self.sigma_g, global_min_pos, local_min_params, self.noise_factor)
        X, Y, Z = terrain_loss.generate_terrain_mesh(x_range, y_range)
        X, Y, Z = X.numpy(), Y.numpy(), Z.numpy()

        # Create 3D mesh plot
        fig_3d = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')])
        fig_3d.update_layout(scene=dict(xaxis_title='X Axis', yaxis_title='Y Axis', zaxis_title='Z Axis'),
                             title="3D Terrain-like Mesh Plot",
                             height=800)  # Adjust the height for 3D plot

        # Create contour plot
        contour = go.Contour(z=Z.flatten(), x=X.flatten(), y=Y.flatten(), colorscale='Viridis', contours=dict(showlabels=True))
        fig_contour = go.Figure(data=[contour])
        fig_contour.update_layout(title="Contour Plot", xaxis_title="X Axis", yaxis_title="Y Axis", height=800)  # Adjust the height for contour plot

        # Optimizer part
        start_x, start_y = self.start_point
        optimizer_class = getattr(optim, self.optimizer)
        terrain_optimizer = TerrainOptimizer(terrain_loss, start_x, start_y, optimizer=optimizer_class, **optim_args)
        trace = terrain_optimizer.optimize(self.log_iter, self.n_iter)

        trace_x, trace_y, trace_z = zip(*[(x, y, terrain_loss.get_loss(torch.tensor([[x]]), torch.tensor([[y]])).item()) for x, y, _ in trace])

        # Add trace to 3D plot with lines to represent movement
        fig_3d.add_trace(go.Scatter3d(x=trace_x, y=trace_y, z=trace_z, mode='markers+lines', marker=dict(size=3, color='red'), name="Optimization Trace"))

        # Add trace to Contour plot with lines for direction
        fig_contour.add_trace(go.Scatter(x=trace_x, y=trace_y, mode='lines+markers', line=dict(color='red'), marker=dict(size=6, color='red'), name="Optimization Trace"))

        return pn.Row(fig_3d, fig_contour)

# Instantiate the TerrainModel
terrain_model = TerrainModel()

# Create Panel dashboard with dynamic controls and live plot update, now with optimizer controls
pn.Row(terrain_model.param, terrain_model.generate_plot).servable()

100%|██████████| 10000/10000 [00:15<00:00, 649.61it/s]


Row
    [0] Column(margin=(5, 10), name='TerrainModel')
        [0] StaticText(value='<b>TerrainModel</b>')
        [1] FloatSlider(end=0, name='X min', start=-20, value=-10)
        [2] FloatSlider(end=20, name='X max', value=10)
        [3] FloatSlider(end=0, name='Y min', start=-20, value=-10)
        [4] FloatSlider(end=20, name='Y max', value=10)
        [5] FloatSlider(end=0, name='A g', start=-20, value=-10)
        [6] FloatSlider(end=5, name='Sigma g', start=0.1, value=2)
        [7] FloatSlider(end=10, name='Global min x', start=-10)
        [8] FloatSlider(end=10, name='Global min y', start=-10)
        [9] TextInput(description='Local minima p..., name='Local min params', value='(-5, 1, 3, 3), ...)
        [10] FloatSlider(name='Noise factor', step=0.01)
        [11] Select(name='Optimizer', options=OrderedDict({'SGD': 'SGD',...]), value='SGD')
        [12] TextInput(description='Optimizer arguments i..., name='Optim args', value='{"lr": 0.1}')
        [13] TupleInput(description='Starting point (x0, y0)', name='Start point', type=<class 'tuple'>, value=(-7.0, 5.0))
        [14] IntSlider(end=10000, name='N iter', start=100, value=10000)
        [15] IntSlider(end=100, name='Log iter', start=10, value=100)
        [16] Button(name='Update Plot')
    [1] ParamMethod(method, _pane=Row, defer_load=False)